In [9]:
%matplotlib
import kwant
from matplotlib import pyplot
import tinyarray
import numpy as np
import time
from numpy import exp,pi,kron,cos,sin,sqrt,pi,cosh,tanh
from scipy import sparse, integrate
import concurrent.futures
import scipy.sparse.linalg as sla

sigma_0 = np.array([[1,0],[0,1]])
sigma_x = np.array([[0,1],[1,0]])
sigma_y = np.array([[0,-1j],[1j,0]])
sigma_z = np.array([[1,0],[0,-1]])

tau_0 = sigma_0
tau_x = sigma_x
tau_y = sigma_y
tau_z = sigma_z

s_0 = sigma_0
s_x = sigma_x
s_y = sigma_y
s_z = sigma_z


Lx = 11
Ly = 11

def make_system():
    
    lat = kwant.lattice.cubic(1,norbs=2)
    sys = kwant.builder.Builder()
    
    def sys_shape(pos):
        x,y,z = pos
        return -Lx<= x <=Lx and -Ly<= y <= Ly and 0<= z <= 1
        
    def lead_shape(pos):
        x,y,z = pos
        return -Ly<= y <= Ly and 0<= z <= 1
    
    def onsite (site,t_dx,t_dy,mu_d_down,mu_d_up,delta_dx_down,delta_dx_up,delta_dy_down,delta_dy_up,phi):
        x,y,z = site.pos
        if z == 0:
            return (2*t_dx+2*t_dy-mu_d_down)*tau_z
        else:
            return (2*t_dx+2*t_dy-mu_d_up)*tau_z
                    
    
    def hopping_x(site1,site2,t_dx,delta_dx_down,delta_dx_up,phi):
        x1,y1,z1 = site1.pos
        x2,y2,z2 = site2.pos
        if z1==0 and z2==0:
            return  -t_dx*tau_z-(delta_dx_down*cos(phi*pi)*tau_x+delta_dx_down*sin(phi*pi)*tau_y)
        else:
            return -t_dx*tau_z-delta_dx_up*tau_x
        
    
    def hopping_y(site1,site2,t_dy,delta_dy_down,delta_dy_up,phi):
        x1,y1,z1 = site1.pos
        x2,y2,z2 = site2.pos
        if z1==0 and z2==0:
            return  -t_dy*tau_z+(delta_dy_down*cos(phi*pi)*tau_x+delta_dy_down*sin(phi*pi)*tau_y)
        else:
            return -t_dy*tau_z+delta_dy_up*tau_x
        
        
    def hopping_z(site1,site2,t_couple):
        return t_couple *tau_z

    
    sys[lat.shape(sys_shape, (0,0,0))] = onsite
    sys[kwant.builder.HoppingKind((1, 0,0), lat, lat)] = hopping_x
    sys[kwant.builder.HoppingKind((0, 1,0), lat, lat)] = hopping_y
    sys[kwant.builder.HoppingKind((0, 0,1), lat, lat)] = hopping_z


    lead = kwant.Builder(kwant.TranslationalSymmetry((1,0,0)))
    lead[lat.shape(sys_shape, (0,0,0))] = onsite
    lead[kwant.builder.HoppingKind((1, 0,0), lat, lat)] = hopping_x
    lead[kwant.builder.HoppingKind((0, 1,0), lat, lat)] = hopping_y
    lead[kwant.builder.HoppingKind((0, 0,1), lat, lat)] = hopping_z
    lead1=lead.reversed()
    sys.attach_lead(lead)
    
    return sys


t_dx = 1
t_dy = 1

mu_d_down = 2.5
mu_d_up = 2.5


p_down = 1
delta_dx_down = 0.2
delta_dy_down = (1-2*p_down)*0.2

p_up = 0
delta_dx_up = 0.2 
delta_dy_up = (1-2*p_up)*0.2

t_couple = 0.4

def plot_ABS(phi):
    
    print(phi)
    
    sys = make_system().finalized()
    
    parameters = dict(t_dx=t_dx,t_dy=t_dy,mu_d_up=mu_d_up,mu_d_down=mu_d_down,delta_dx_down=delta_dx_down,delta_dx_up=delta_dx_up,\
                      delta_dy_down=delta_dy_down,delta_dy_up=delta_dy_up,phi=phi,t_couple=t_couple) 
    
    ham_mat = sys.hamiltonian_submatrix(sparse=True,params = parameters)
    ev = sla.eigsh(ham_mat, k=200, which = 'SA',sigma=0.00000001, return_eigenvectors=False)
    evs = np.sort(ev,axis=0)
    
    return evs

# def plot_wf(phi_s):
    
#     sys = make_system().finalized()

#     dim = len(sys.sites)*4/2
#     print("number of E<0",dim)
#     parameters = dict(t_s=t_s,mu_s=mu_s,delta_s=delta_s,t_dx=t_dx,t_dy=t_dy,mu_d=mu_d,delta_dx=delta_dx,\
#                       delta_dy=delta_dy,phi_s=phi_s,t_couple=t_couple) 
    
#     ham_mat = sys.hamiltonian_submatrix(sparse=True,params = parameters)
# #     ev, ewf = sla.eigsh(ham_mat, k=dim, which = 'SA',sigma=0.00000001, return_eigenvectors=True)
# # # which = 'LM'
# #     ## False
# #     print(np.shape(ev))
# #     print(ev)

#     ev_o = sla.eigsh(ham_mat, k=dim, which = 'SA',sigma=0.00000001, return_eigenvectors=False)
#     print(np.shape(ev_o))
#     print(ev_o)


#     # rho = kwant.operator.Density(sys)
#     # den = rho(ewf[:,1])
#     # a = 0
#     # b = 0
#     # for i in range(len(sys.sites)):
#     #     if sys.sites[i].pos[2]>0:
#     #         a = a + den[i]
#     #     else:
#     #         b = b + den[i]
#     # print(a,b)


# def family_color(site):
    
#     x,y,z = site.pos
#     if z==0:
#         return  'red' ## s-wave
#     if z==1:
#         return 'blue' ## d-wave


def main():
    start = time.time() 


    
    # sys = make_system()
    # kwant.plot(sys, site_size=0.18, site_lw=0.01, hop_lw=0.05,site_color=family_color)

    # plot_band(0.)
    # plot_wf(0.5001)

    
    sys = make_system().finalized()
 
    phidif = np.linspace(0,2,21)
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=7) as executor:
        bb=executor.map(plot_ABS,phidif)  
    Ea = list(bb)
    
    pyplot.figure()
    pyplot.plot(phidif,Ea,'dodgerblue')
    pyplot.xlabel(r'$\phi/\pi$')
    pyplot.ylabel(r'$E$')
    pyplot.xlim(0,2)
#     pyplot.ylim(-1,1)
    
    E = np.array(Ea)
    Ef = E.sum(axis=1)
#     c2 = (Ef[0]-Ef[5])/2
#     print(("Coefficient of cos(2phi)",c2))
    
    pyplot.figure()
    pyplot.plot(phidif,Ef,'dodgerblue')
    pyplot.xlabel(r'$\phi/\pi$')
    pyplot.ylabel(r'$E$')
#     pyplot.xlim(0,2)
    pyplot.title("Free energy")
    
    

    
    end = time.time()     
    print('Running time: %s seconds'%(end-start))
# Call the main function if the script gets executed (as opposed to imported).
# See <http://docs.python.org/library/__main__.html>.
if __name__ == '__main__':
    main()
    


Using matplotlib backend: Qt5Agg
0.2
0.0
0.1
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0.9
1.0
1.1
1.2000000000000002
1.3
1.4000000000000001
1.5
1.6
1.7000000000000002
1.8
1.9000000000000001
2.0
Running time: 181.68956589698792 seconds
